# Why a tiny model ?
For several real time applications that require the model to run local and every second, The accurate resnet architectures fail ! because they are very large models 


# Step 1 -> Load needed libraries

In [1]:
# for working with files 
import glob
import os
import shutil
import itertools  
from tqdm import tqdm

# for working with images
from PIL import Image
import numpy as np
import pandas as pd
from skimage import transform
import matplotlib.pyplot as plt
import cv2 as cv
import scipy.io
import random

# tensorflow stuff
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout, BatchNormalization, GlobalAveragePooling2D, Add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers, optimizers, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.activations import relu, softmax



# for evaluation
from sklearn.metrics import classification_report, confusion_matrix

2023-03-24 17:01:07.036217: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 17:01:07.414609: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-24 17:01:21.477838: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rparchiev/.local/lib/python3.10/site-packages/cv2/../../lib64::/usr/local/lib:/us

2.11.0


# Step 2 -> Image preprocessing
## 1. New directories
Make new directories to store the preprocessed images

In [2]:
!mkdir car_data_cropped/
!mkdir car_data_cropped/train
!mkdir car_data_cropped/test

/bin/bash: /home/rparchiev/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/rparchiev/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/rparchiev/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## 2. Crop the images in the training folder
Thanks to Stanford, they provide crop dimensions for the car in each photo 

In [3]:
cars_annos = pd.read_csv('train_dataset_denseNet/anno_train.csv',header=None)
fnames = []
class_ids = []
bboxes = []
labels = []


for i in range(len(cars_annos)):
    annotation = cars_annos.iloc[i]
    bbox_x1 = annotation[1]
    bbox_y1 = annotation[2]
    bbox_x2 = annotation[3]
    bbox_y2 = annotation[4]
    class_id = annotation[5]
    labels.append('%04d' % (class_id,))
    fname = annotation[0]
    bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
    class_ids.append(class_id)
    fnames.append(fname)


l = glob.glob('train_dataset_denseNet/car_data/car_data/train/*/*')

for j in tqdm(range(len(l))):
    i = fnames.index(l[j].split('/')[-1])
    labels[i]
    (x1, y1, x2, y2) = bboxes[i]
    fname=l[j].split('/')[-1]
    class_name = l[j].split('/')[-2]
    src_path = os.path.join('train_dataset_denseNet/car_data/car_data/train/'+class_name+'/', fname)
    src_image = cv.imread(src_path)

    height, width = src_image.shape[:2]

    # margins of 16 pixels
    margin = 16
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(x2 + margin, width)
    y2 = min(y2 + margin, height)
    # print("{} -> {}".format(fname, label))


    dst_path = os.path.join('car_data_cropped/train/', class_name)
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)


    dst_path = os.path.join(dst_path, fname)


    crop_image = src_image[y1:y2, x1:x2]
    #dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))
    cv.imwrite(dst_path, crop_image)

100%|███████████████████████████████████████| 8144/8144 [02:47<00:00, 48.72it/s]


## 3. Crop the images in the test directory
Here I will use them as validation data 

In [4]:
cars_annos = pd.read_csv('train_dataset_denseNet/anno_test.csv',header=None)
fnames = []
class_ids = []
bboxes = []
labels = []


l = glob.glob('train_dataset_denseNet/car_data/car_data/test/*/*')


for i in range(len(cars_annos)):
    annotation = cars_annos.iloc[i]
    bbox_x1 = annotation[1]
    bbox_y1 = annotation[2]
    bbox_x2 = annotation[3]
    bbox_y2 = annotation[4]
    class_id = annotation[5]
    labels.append('%04d' % (class_id,))
    fname = annotation[0]
    bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
    class_ids.append(class_id)
    fnames.append(fname)


In [5]:
for j in tqdm(range(len(l))):
    i = fnames.index(l[j].split('/')[-1])

    (x1, y1, x2, y2) = bboxes[i]
    fname=l[j].split('/')[-1]

    class_name = l[j].split('/')[-2]
    src_path = os.path.join('train_dataset_denseNet/car_data/car_data/test/'+class_name+'/', fname)
    src_image = cv.imread(src_path)

    height, width = src_image.shape[:2]

    # margins of 16 pixels
    margin = 16
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(x2 + margin, width)
    y2 = min(y2 + margin, height)
    # print("{} -> {}".format(fname, label))


    dst_path = os.path.join('car_data_cropped/test/', class_name)
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)

    dst_path = os.path.join(dst_path, fname)


    crop_image = src_image[y1:y2, x1:x2]
    #dst_img = cv.resize(src=crop_image, dsize=(img_height, img_width))

    cv.imwrite(dst_path, crop_image)

100%|███████████████████████████████████████| 8041/8041 [02:41<00:00, 49.88it/s]


# Step 3 -> prepare the image generators

In [5]:
attempt = 1
if attempt == 0:
    train_datagen=ImageDataGenerator(rotation_range=20,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     zoom_range=0.2,
                                     horizontal_flip=True)


    valid_datagen=ImageDataGenerator(rotation_range=20,
                                    zoom_range=0.15,
                                    horizontal_flip=True)


    train_generator=train_datagen.flow_from_directory(
        directory="car_data_cropped/train/",
        batch_size=64,
        seed=42,
        target_size=(224,224))


    valid_generator=valid_datagen.flow_from_directory(
        directory="car_data_cropped/test/",
        batch_size=64,
        seed=42,
        target_size=(224,224))

#--------------------------------------------------------------------------
if attempt == 1:
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

    train_datagen=ImageDataGenerator(rotation_range=15,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     preprocessing_function=preprocess_input)

    valid_datagen=ImageDataGenerator(horizontal_flip=True, 
                                     preprocessing_function=preprocess_input)


    train_generator=train_datagen.flow_from_directory(
        directory="car_data_cropped/train/",
        batch_size=64,
        seed=42,
        target_size=(224,224))


    valid_generator=valid_datagen.flow_from_directory(
        directory="car_data_cropped/test/",
        batch_size=300,
        seed=42,
        target_size=(224,224))


Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


# <font color='red'> Attempt 0: Build a tiny resnet from scratch </font>

In [7]:
def block(n_output, upscale=False):
    # n_output: number of feature maps in the block
    # upscale: should we use the 1x1 conv2d mapping for shortcut or not
    
    # keras functional api: return the function of type
    # Tensor -> Tensor
    
    def f(x):
        
        # H_l(x):
        # first pre-activation
        h = BatchNormalization()(x)
        h = Activation(relu)(h)
        # first convolution
        h = Conv2D(kernel_size=3, filters=n_output, strides=1, padding='same', kernel_regularizer=regularizers.l2(0.01))(h)
        
        # second pre-activation
        h = BatchNormalization()(x)
        h = Activation(relu)(h)
        # second convolution
        h = Conv2D(kernel_size=3, filters=n_output, strides=1, padding='same', kernel_regularizer=regularizers.l2(0.01))(h)
        
        # f(x):
        if upscale:
            # 1x1 conv2d
            f = Conv2D(kernel_size=1, filters=n_output, strides=1, padding='same')(x)
        else:
            # identity
            f = x
        
        # F_l(x) = f(x) + H_l(x):
        return Add()([f, h])
    
    return f

In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy()

2023-03-24 17:02:35.769857: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 17:02:37.090643: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78971 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0
2023-03-24 17:02:37.091938: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78971 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:98:00.0, compute capability: 8.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [12]:
with mirrored_strategy.scope():
    # input tensor is the 28x28 grayscale image
    input_tensor = Input((224, 224, 3))

    # first conv2d with post-activation to transform the input data to some reasonable form
    x = Conv2D(kernel_size=3, filters=16, strides=1, padding='same', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(relu)(x)

    # F_1
    x = block(16)(x)
    # F_2
    x = block(16)(x)

    # F_3
    # H_3 is the function from the tensor of size 28x28x16 to the the tensor of size 28x28x32
    # and we can't add together tensors of inconsistent sizes, so we use upscale=True
    x = block(32, upscale=True)(x)       # !!! <------- Uncomment for local evaluation
    # F_4
    x = block(32)(x)                     # !!! <------- Uncomment for local evaluation
    # F_5
    x = block(32)(x)                     # !!! <------- Uncomment for local evaluation

    # F_6
    x = block(48, upscale=True)(x)       # !!! <------- Uncomment for local evaluation
    # F_7
    x = block(48)(x)                     # !!! <------- Uncomment for local evaluation

    # last activation of the entire network's output
    x = BatchNormalization()(x)
    x = Activation(relu)(x)

    # average pooling across the channels
    # 28x28x48 -> 1x48
    x = GlobalAveragePooling2D()(x)

    # dropout for more robust learning
    x = Dropout(0.2)(x)

    # last softmax layer
    x = Dense(units=196, kernel_regularizer=regularizers.l2(0.01))(x)
    x = Activation(softmax)(x)
    model = Model(inputs=input_tensor, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
len(model.layers)

40

In [14]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_68 (Conv2D)             (None, 224, 224, 16  448         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_64 (BatchN  (None, 224, 224, 16  64         ['conv2d_68[0][0]']              
 ormalization)                  )                                                           

In [16]:
reduce_lr = ReduceLROnPlateau('val_acc', factor=0.1, patience=1, verbose=1)

model.fit(train_generator,
          steps_per_epoch=100,
          validation_data=(x_val,y_val),
          epochs=40,verbose=1)

2023-03-24 16:06:42.318196: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\017TensorDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



Epoch 1/40
INFO:tensorflow:batch_all_reduce: 40 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 40 all-reduces with algorithm = nccl, num_packs = 1


2023-03-24 16:06:53.634873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-24 16:06:55.211604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-24 16:06:58.727448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-24 16:07:00.343747: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55e45c0b8e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-24 16:07:00.343782: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-03-24 16:07:00.343789: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-03-24 16:07:00.353817: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_

100/100 [==============================] - 96s 738ms/step - loss: 6.6017 - accuracy: 0.0075 - val_loss: 5.7559 - val_accuracy: 0.0067
Epoch 2/40
100/100 [==============================] - 72s 721ms/step - loss: 5.4946 - accuracy: 0.0112 - val_loss: 5.4248 - val_accuracy: 0.0133
Epoch 3/40
100/100 [==============================] - 73s 726ms/step - loss: 5.3053 - accuracy: 0.0151 - val_loss: 5.3417 - val_accuracy: 0.0133
Epoch 4/40
100/100 [==============================] - 72s 719ms/step - loss: 5.2490 - accuracy: 0.0192 - val_loss: 5.2941 - val_accuracy: 0.0200
Epoch 5/40
100/100 [==============================] - 71s 704ms/step - loss: 5.2295 - accuracy: 0.0183 - val_loss: 5.2464 - val_accuracy: 0.0233
Epoch 6/40
100/100 [==============================] - 70s 698ms/step - loss: 5.2130 - accuracy: 0.0187 - val_loss: 5.2334 - val_accuracy: 0.0167
Epoch 7/40
 65/100 [==================>...........] - ETA: 24s - loss: 5.2015 - accuracy: 0.0199

KeyboardInterrupt: 

# <font> Очень долго сходится, поэтому перейдем к mobilenet</font>

# <font color='red'> Attempt 2: use mobilenet</font>

In [6]:
for x,y in valid_generator:
    x_val = x
    y_val = y
    break;

In [7]:
IMAGE_SIZE = 224
# Base model with MobileNetV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)


with mirrored_strategy.scope():
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,alpha = .5,
                                                   include_top=False, 
                                                   weights='imagenet')

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = Dropout(.6)(x)
    prediction_layer = tf.keras.layers.Dense(196, activation='softmax')(x)

    learning_rate = 0.0001

    model=Model(inputs=base_model.input,outputs=prediction_layer)

    for layer in model.layers[:80]:
        layer.trainable=False
    for layer in model.layers[80:]:
        layer.trainable=True
    # 

    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,clipnorm=0.001)
    def get_lr_metric(optimizer):
        def lr(y_true, y_pred):
            return optimizer.lr
        return lr

    lr_metric = get_lr_metric(optimizer)

    model.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics=['accuracy',lr_metric])


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [8]:

reduce_lr = ReduceLROnPlateau('val_acc', factor=0.1, patience=1, verbose=1)

model.fit(train_generator,
          steps_per_epoch=100,
          validation_data=(x_val,y_val),
          epochs=40,verbose=1)



2023-03-24 17:04:41.696929: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\017TensorDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



Epoch 1/40
INFO:tensorflow:batch_all_reduce: 77 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 77 all-reduces with algorithm = nccl, num_packs = 1


2023-03-24 17:05:03.904777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-24 17:05:05.430918: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-24 17:05:08.686829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-24 17:05:09.581379: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f7d8824a4a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-24 17:05:09.581410: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-03-24 17:05:09.581416: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2023-03-24 17:05:09.587105: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_

100/100 [==============================] - 113s 781ms/step - loss: 5.7336 - accuracy: 0.0077 - lr: 1.0000e-04 - val_loss: 5.3532 - val_accuracy: 0.0200 - val_lr: 1.0000e-04
Epoch 2/40
100/100 [==============================] - 78s 778ms/step - loss: 5.3084 - accuracy: 0.0162 - lr: 1.0000e-04 - val_loss: 5.1244 - val_accuracy: 0.0467 - val_lr: 1.0000e-04
Epoch 3/40
100/100 [==============================] - 80s 793ms/step - loss: 5.0361 - accuracy: 0.0346 - lr: 1.0000e-04 - val_loss: 4.8430 - val_accuracy: 0.0600 - val_lr: 1.0000e-04
Epoch 4/40
100/100 [==============================] - 77s 771ms/step - loss: 4.7502 - accuracy: 0.0612 - lr: 1.0000e-04 - val_loss: 4.5534 - val_accuracy: 0.0933 - val_lr: 1.0000e-04
Epoch 5/40
100/100 [==============================] - 77s 764ms/step - loss: 4.4490 - accuracy: 0.0898 - lr: 1.0000e-04 - val_loss: 4.2374 - val_accuracy: 0.1167 - val_lr: 1.0000e-04
Epoch 6/40
100/100 [==============================] - 90s 896ms/step - loss: 4.1560 - accuracy:

# Step 4 -> Evaluate the model

In [9]:
scoreSeg = model.evaluate_generator(valid_generator)
print("Accuracy = ",scoreSeg[1])

for i,j in valid_generator:
    print(i.shape, j.shape)
    p = model.predict(i)
    p = p.argmax(-1)
    t = j.argmax(-1)
    print(classification_report(t,p))
    print(confusion_matrix(t,p))
    break;


/tmp/ipykernel_125096/754504181.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scoreSeg = model.evaluate_generator(valid_generator)
2023-03-24 17:57:16.218508: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021TensorDataset:875"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
      

Accuracy =  0.7739087343215942
(300, 224, 224, 3) (300, 196)
10/10 [==============================] - 3s 16ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.00      0.00      0.00         1
           2       0.50      1.00      0.67         1
           3       1.00      1.00      1.00         2
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      0.50      0.67         2
           8       1.00      1.00      1.00         2
           9       1.00      0.50      0.67         2
          10       1.00      1.00      1.00         1
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         2
          13       0.33      1.00      0.50         1
          14       1.00      1.00      1.00         1
          15       1

/home/rparchiev/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rparchiev/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rparchiev/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rparchiev/.local/lib/python3.1

In [10]:
p

array([ 22,  33, 106, 160,  33, 138, 173,  29, 111,  92,  18, 165,  49,
       174,  77, 150, 177,  97,  47,  73, 107,  63, 137, 143,  64, 102,
       181, 172, 122,   2, 148,  58,   8,  13, 171, 132,  22, 152,  81,
        66,  16,   1, 127,  52, 143, 110,  65, 141, 192, 127,  13, 165,
        71, 101,  59, 160,  58,   2, 192,  34, 159,  49,  43, 190, 103,
        47, 103, 109, 104,  61, 161, 119, 114, 176,  73, 149, 147, 195,
       119,  13,  22,  58,   8, 195,  21,  63, 174,  96, 176,  92, 156,
        10, 145,   3, 147, 154,  33,   0,  81, 133, 164, 101, 110, 194,
        91, 151,  33, 174,  63, 150, 160, 169, 152, 138, 168, 119,  71,
        20, 107, 121, 114, 162,  33,  93,   3,  46, 157,  18,  94, 138,
        67,  63,  50, 109, 141,  33, 140, 194,  75,  65,  56, 103,  48,
       177, 103, 150, 103,  85, 131, 160, 107,  55,  77,  22,  29,  49,
        27, 100, 192, 173,  48, 126, 127,  74, 163, 154,  47,  67, 146,
       192, 161, 176, 101,  56, 177,  43, 192,  26,  11,  39, 15

In [11]:
{i:j for j,i in valid_generator.class_indices.items()}

{0: 'AM General Hummer SUV 2000',
 1: 'Acura Integra Type R 2001',
 2: 'Acura RL Sedan 2012',
 3: 'Acura TL Sedan 2012',
 4: 'Acura TL Type-S 2008',
 5: 'Acura TSX Sedan 2012',
 6: 'Acura ZDX Hatchback 2012',
 7: 'Aston Martin V8 Vantage Convertible 2012',
 8: 'Aston Martin V8 Vantage Coupe 2012',
 9: 'Aston Martin Virage Convertible 2012',
 10: 'Aston Martin Virage Coupe 2012',
 11: 'Audi 100 Sedan 1994',
 12: 'Audi 100 Wagon 1994',
 13: 'Audi A5 Coupe 2012',
 14: 'Audi R8 Coupe 2012',
 15: 'Audi RS 4 Convertible 2008',
 16: 'Audi S4 Sedan 2007',
 17: 'Audi S4 Sedan 2012',
 18: 'Audi S5 Convertible 2012',
 19: 'Audi S5 Coupe 2012',
 20: 'Audi S6 Sedan 2011',
 21: 'Audi TT Hatchback 2011',
 22: 'Audi TT RS Coupe 2012',
 23: 'Audi TTS Coupe 2012',
 24: 'Audi V8 Sedan 1994',
 25: 'BMW 1 Series Convertible 2012',
 26: 'BMW 1 Series Coupe 2012',
 27: 'BMW 3 Series Sedan 2012',
 28: 'BMW 3 Series Wagon 2012',
 29: 'BMW 6 Series Convertible 2007',
 30: 'BMW ActiveHybrid 5 Sedan 2012',
 31: '

In [12]:
t

array([ 21,  29, 106,  25,  33, 138, 173,  29, 137,  92,  18,  96,  49,
       176,  77, 150, 177,  97,  47,  73, 107,  65, 137, 143,  62, 103,
       181, 172, 122,   2, 148,  57,   8,  28, 171, 132,  21, 152,  81,
        66,  16, 171, 127,  52, 143, 110,  65, 141, 190, 127,  19, 165,
       126, 101, 119, 160,  58, 126, 192,  34, 159,  49,  38, 190, 103,
        47, 103, 109, 104,  61, 161,  59, 114, 176,  73, 149, 146, 195,
        81,  13,  21,   7,   8, 195,  23,  62, 174,  96, 176,  92, 156,
        10, 145,   3, 147, 153,  33,   0,  81, 133, 164, 103, 110, 194,
        91, 151,   9,   4,  39, 150, 115, 169, 157, 138, 168, 119,  71,
        20, 107, 121, 114, 162,  33,  81,   3,  46, 157,  18,  94, 138,
       118,  63,  50, 109, 141,  33, 140, 194,  75,  97, 175, 103,  48,
       177, 103,   1, 103,  85, 131, 160, 107,  55,  77,  19,  29,  49,
        27, 100, 147, 173,  48, 126, 173,  74, 163, 153,  47,  68, 145,
       192, 163, 176, 101,  56, 177,  43, 192,  26,  12,  39,  4

In [ ]:
tf.keras.models.save_model(
    model,
    "cars1_half.h5"
)


In [ ]:
1 /196

In [ ]:
ls -la

In [ ]:
from IPython.display import FileLink
FileLink('cars1_half2.h5')

In [13]:
model.save('saved_models/MobileNet_77acc')

INFO:tensorflow:Assets written to: saved_models/MobileNet_77acc/assets


INFO:tensorflow:Assets written to: saved_models/MobileNet_77acc/assets
